## Packages

In [4]:
import pandas as pd
import re
import string
import nltk
from sklearn.manifold import TSNE
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

## Functions

In [5]:
def transformDocument(df, column_name, language):
    stop_words = usingStopwords(language)
    df_pp = df.copy()
    # 1. Aplicar preprocessamento nos títulos e textos completos
    if language == 'pt':
        # Substituir símbolos importantes
        df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('-feira', ''))
        df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('+', 'mais '))
        df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('-', 'menos '))
        df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('%', ' por cento'))
        df_pp[column_name] = df_pp[column_name].map(lambda s: removeStopwords(s, stop_words))

    elif language == 'en':
        df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('-', 'less'))
        df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('+', 'plus '))
        df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('%', ' percent'))
        df_pp[column_name] = df_pp[column_name].map(lambda s: removeStopwords(s, stop_words))

    else:
        pass

    df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('R$', ''))
    df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('U$', ''))
    df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('US$', ''))
    df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('S&P 500', 'spx'))

    # Transformar em String e Letras Minúsculas nas Mensagens
    df_pp[column_name] = df_pp[column_name].map(lambda s:
                                              normalizarString(s))


    # Remover Pontuações
    # Remover Pontuações
    df_pp[column_name] = df_pp[column_name].map(lambda s: s.translate(str.maketrans('', '', string.punctuation)))

    # Remover Emojis
    df_pp[column_name] = df_pp[column_name].map(lambda s: removeEmojis(s))

    # Quebras de Linha desnecessárias
    df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('\n', ' '))

    # Remover aspas duplas
    df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('\"', ''))
    df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('“', ''))
    df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('”', ''))

    # Remover valores
    df_pp[column_name] = df_pp[column_name].map(lambda s: removeValores(s))

    # Espaços desnecessários
    df_pp[column_name] = df_pp[column_name].map(lambda s: s.strip())
    return df_pp



def removeEmojis(sentence):
    "Remoção de Emojis nas mensagens de texto."

    # Padrões dos Emojis
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u'\U00010000-\U0010ffff'
                               u"\u200d"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\u3030"
                               u"\ufe0f"
                               "]+", flags=re.UNICODE)

    return emoji_pattern.sub(r'', sentence)

def removeValores(sentence):
    new_sentece = ''

    for token in sentence.split():
        if token.isdigit():
            token = '<NUM>'
        new_sentece += ' {}'.format(token)

    return new_sentece

def usingStopwords(language):
    stop_words = []

    nltk.download('stopwords')

    if language == 'pt':
        stop_words = nltk.corpus.stopwords.words('portuguese')
    elif language == 'en':
        stop_words = nltk.corpus.stopwords.words('english')

    return stop_words

def removeStopwords(text, stop_words):
    tokens = []
    for word in text.split():
        if word not in stop_words:
            tokens.append(word)

    text = ' '.join(tokens)
    return text

def normalizarString(text):
    """
    Função para retirar acentuações e converter para minúscula
    :param text:
    :return text_normalizado
    """
    import unicodedata

    text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode("utf-8")
    return str(text.lower())

## Dependencies

In [6]:
train_raw = pd.read_csv('../assets/data/splits/train/raw.csv')
val_raw = pd.read_csv('../assets/data/splits/val/raw.csv')
test_raw = pd.read_csv('../assets/data/splits/test/raw.csv')
df = pd.read_csv('../assets/data/df.csv')
df

,title,tags,url,time,label
0,OGCI estabelece nova meta de emissão de carbon...,"['Combustíveis', 'Empresas', 'Meio Ambiente', ...",https://www.moneytimes.com.br/ogci-estabelece-...,10:15:00,0
1,Pelo acréscimo previsto da nova gasolina e efi...,"['Agronegócio', 'Etanol', 'Gasolina', 'Petrobr...",https://www.moneytimes.com.br/pelo-acrescimo-p...,10:35:00,0
2,"Ibovespa hoje cai 1,22% em mau humor com bater...","['Ibovespa', 'JHSF (JHSF3)', 'Petrobras (PETR4...",https://www.suno.com.br/noticias/ibovespa-fech...,17:28:00,0
3,Petrobras (PETR4) conclui venda de campos pelo...,"['PETR3', 'Petrobras (PETR4)']",https://www.suno.com.br/noticias/petrobras-pet...,09:46:00,0
4,ANP autoriza Petrobras a movimentar gasolina d...,"['Aeroportos', 'Agência Brasil', 'Anac', 'ANP'...",https://www.moneytimes.com.br/anp-autoriza-pet...,14:06:00,0
...,...,...,...,...,...
2032,Ibovespa tem nova queda com mercado à espera d...,"['Ações', 'Alliar', 'B3', 'Braskem', 'BRF', 'E...",https://www.moneytimes.com.br/ibovespa-estende...,10:15:00,0
2033,Discurso mais duro do Fed leva Ibovespa a maio...,"['3R Petroleum', 'Banco Inter', 'Bradespar', '...",https://www.moneytimes.com.br/ibovespa-acelera...,17:15:00,0
2034,Ações ligadas a commodities dominam lista de i...,"['Ações', 'Ágora Investimentos', 'Ativa Invest...",https://www.moneytimes.com.br/acoes-ligadas-a-...,17:40:00,0
2035,"Após alta de 5,5%, Mirae faz 3 substituições e...","['Ações', 'AES Brasil', 'Carteira Recomendada'...",https://www.moneytimes.com.br/apos-alta-de-55-...,15:01:00,0


## Preprocessing splits

In [7]:
train_pp = transformDocument(train_raw, 'title', 'pt')
val_pp = transformDocument(val_raw, 'title', 'pt')
test_pp = transformDocument(test_raw, 'title', 'pt')
df_pp = transformDocument(df, 'title', 'pt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Guilherme\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Guilherme\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Guilherme\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Guilherme\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
df_pp

,title,tags,url,time,label
0,ogci estabelece nova meta emissao carbono afir...,"['Combustíveis', 'Empresas', 'Meio Ambiente', ...",https://www.moneytimes.com.br/ogci-estabelece-...,10:15:00,0
1,pelo acrescimo previsto nova gasolina eficienc...,"['Agronegócio', 'Etanol', 'Gasolina', 'Petrobr...",https://www.moneytimes.com.br/pelo-acrescimo-p...,10:35:00,0
2,ibovespa hoje cai <NUM> cento mau humor bateri...,"['Ibovespa', 'JHSF (JHSF3)', 'Petrobras (PETR4...",https://www.suno.com.br/noticias/ibovespa-fech...,17:28:00,0
3,petrobras petr4 conclui venda campos valor tot...,"['PETR3', 'Petrobras (PETR4)']",https://www.suno.com.br/noticias/petrobras-pet...,09:46:00,0
4,anp autoriza petrobras movimentar gasolina avi...,"['Aeroportos', 'Agência Brasil', 'Anac', 'ANP'...",https://www.moneytimes.com.br/anp-autoriza-pet...,14:06:00,0
...,...,...,...,...,...
2032,ibovespa nova queda mercado espera ata fed,"['Ações', 'Alliar', 'B3', 'Braskem', 'BRF', 'E...",https://www.moneytimes.com.br/ibovespa-estende...,10:15:00,0
2033,discurso duro fed leva ibovespa maior tombo de...,"['3R Petroleum', 'Banco Inter', 'Bradespar', '...",https://www.moneytimes.com.br/ibovespa-acelera...,17:15:00,0
2034,acoes ligadas commodities dominam lista indica...,"['Ações', 'Ágora Investimentos', 'Ativa Invest...",https://www.moneytimes.com.br/acoes-ligadas-a-...,17:40:00,0
2035,apos alta <NUM> cento mirae faz <NUM> substitu...,"['Ações', 'AES Brasil', 'Carteira Recomendada'...",https://www.moneytimes.com.br/apos-alta-de-55-...,15:01:00,0


## Outputs for EDA, word2vec and preprocessing

In [9]:
train_pp.to_csv('../assets/data/splits/train/preprocessed.csv', index=False)
val_pp.to_csv('../assets/data/splits/val/preprocessed.csv', index=False)
test_pp.to_csv('../assets/data/splits/test/preprocessed.csv', index=False)
df_pp.to_csv('../assets/data/df_pp.csv', index=False)